In [1]:
# decoding JPEG images and decoding/encoding RLE datasets
# !pip3 install pylibjpeg==1.4.0
# https://github.com/pydicom/pylibjpeg

# !pip3 install python-gdcm

In [2]:
DEBUG = False

import os
import sys

In [3]:
# suitable for kaggle notebook
# sys.path = ['../ca_2',] + sys.path
# print(sys.path)

In [4]:
import argparse
import warnings

In [5]:
import gc, ast, cv2, time, pickle, random
# import pylibjpeg
# import gdcm
# import pydicom
# pydicom is a pure Python package for working with DICOM files. 
# -It lets you read, modify and write DICOM data in an easy "pythonic" way. 

In [6]:
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image


# import nibabel as nib
# read / write access to some common neuroimaging file formats

In [7]:
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold

import albumentations # python library for pixel-level augmentations 

In [8]:
%matplotlib inline

In [9]:
import timm

# import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset


/home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [10]:
from tqdm import tqdm

In [11]:
# import graphviz

In [12]:
# # pip3 install torchview
# from torchview import draw_graph

In [13]:
np.set_printoptions(threshold=sys.maxsize)

In [14]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', None) # 500
pd.set_option('expand_frame_repr', True)

In [15]:

device = torch.device('cuda')

# benchmark mode is good whenever your input sizes for your network do not vary. 
# This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.
torch.backends.cudnn.benchmark = True

# Config

In [16]:
kernel_type = '0920_2d_lstmv22headv2_convnn_224_15_6ch_8flip_augv2_drl3_rov1p2_rov3p2_bs4_lr6e5_eta6e6_lw151_50ep'
load_kernel = None
load_last = True

n_folds = 5
backbone = 'convnext_nano'

image_size = 224
n_slice_per_c = 15
in_chans = 6

init_lr = 19e-8 # 19e-5, 
eta_min = 11e-8 # 11e-6
lw = [15, 1]
batch_size = 1
drop_rate = 0. # 0.
drop_path_rate = 0.
drop_rate_last = 0.3 # 0.3
p_mixup = 0.5
p_rand_order = 0.2
p_rand_order_v1 = 0.2

data_dir = './numpy_1/'
use_amp = True
num_workers = 12 # 12
out_dim = 1

n_epochs = 25

log_dir = './logs'
model_dir = './models'
model_dir_seg = './kaggle'
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [17]:
# Albumentations is a computer vision tool that boosts the performance of deep convolutional neural networks.
# Albumentations is a Python library for image augmentation.
transforms_train = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.RandomBrightnessContrast(brightness_limit=0.1, p=0.7),
    albumentations.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, border_mode=4, p=0.7),

    albumentations.OneOf([
        albumentations.MotionBlur(blur_limit=3),
        albumentations.MedianBlur(blur_limit=3),
        albumentations.GaussianBlur(blur_limit=3),
        albumentations.GaussNoise(var_limit=(3.0, 9.0)),
    ], p=0.5),
    albumentations.OneOf([
        albumentations.OpticalDistortion(distort_limit=1.),
        albumentations.GridDistortion(num_steps=5, distort_limit=1.),
    ], p=0.5),

    albumentations.CoarseDropout(max_height=int(image_size * 0.5), max_width=int(image_size * 0.5), max_holes=1, p=0.5),
])

transforms_valid = albumentations.Compose([
    albumentations.Resize(image_size, image_size),
])

/home/na/miniconda3/envs/base_2/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


# DataFrame

In [18]:
df = pd.read_csv('train_seg.csv')
df = df.sample(16).reset_index(drop=True) if DEBUG else df

df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,mask_file,image_folder,w,h,d,t,fold
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,NaN,/data/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200,512,512,243,1.000,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,NaN,/data/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.27262,512,512,406,0.500,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,NaN,/data/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.21561,512,512,385,0.625,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,NaN,/data/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.12351,512,512,501,0.600,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/data/rsna-2022-cervical-spine-fracture-detection/segmentations/1.2.826.0.1.3680043.1363.nii,/data/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.1363,512,512,199,1.000,0


# Dataset

In [19]:
class CLSDataset(Dataset):
    def __init__(self, df, mode, transform):

        self.df = df.reset_index()
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        images_lst = []
        
        tmp = list(range(7))
        ### random order v3
        if self.mode == 'train' and random.random() < p_rand_order:
            random.shuffle(tmp)
        ###
        for cid in (tmp):                
            filepath = os.path.join(data_dir, f'numpy_1/{row.StudyInstanceUID}_{cid+1}.npy')
            images = np.load(filepath)            
                # type(image), image.shape => <class 'numpy.ndarray'> (15, 224, 224, 6)

            images = np.stack([self.transform(image=images[i])['image'] for i in range(n_slice_per_c)], 0)

            images = images.transpose(0,3,1,2)
                # type(image), image.shape => <class 'numpy.ndarray'> (15, 6, 224, 224)        

            images = images / 255. # trim the 'data values' between 0. and 1. 
                # prior to 255. divide, convert data to float       
            images_lst.append(images)                
            
        images_lst = np.stack(images_lst, 0)#.astype(np.float32)

        images_lst = images_lst.reshape((105,6,224,224))

        if self.mode != 'test':
            labels = []
            # tmp => [0, 1, 2, 3, 4, 5, 6]
            for i in row[[f'C{x+1}' for x in tmp]].tolist():
                labels += [i] * n_slice_per_c
                
            # labels => [1, 1, 1, 1, 1, 1, 1, 1, .........., 0, 0, 0, 0, 0, 0, 0, 0] => 105 items

            images = torch.tensor(images_lst).float()
            labels = torch.tensor(labels).float()
            
            if self.mode == 'train' and random.random() < p_rand_order_v1:
                indices = torch.randperm(images.size(0))
                images = images_lst[indices]
                labels = labels[indices]

            return images, labels
        else:
            return torch.tensor(images).float()

In [20]:
# df_show = df[0:1]
# dataset_show = CLSDataset(df_show, 'train', transform=transforms_train)

In [21]:
# aa, bb = dataset_show[0]

# Model

In [22]:
class TimmModelType2(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super(TimmModelType2, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=in_chans,
            num_classes=out_dim,
            features_only=False,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            pretrained=pretrained
        )

        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.InstanceNorm1d(256), # replaced BatchNorm1d for training with batch_size = 1
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, out_dim),
        )
        self.lstm2 = nn.LSTM(hdim, 256, num_layers=2, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.head2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.InstanceNorm1d(256), # replaced BatchNorm1d for training with batch_size = 1
            nn.Dropout(drop_rate_last),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )



    def forward(self, x):  # (bs, nc*7, ch, sz, sz)
        bs = x.shape[0]

        x = x.view(bs * n_slice_per_c * 7, in_chans, image_size, image_size)

        feat = self.encoder(x)
        feat = feat.view(bs, n_slice_per_c * 7, -1)
        feat1, _ = self.lstm(feat)
        feat1 = feat1.contiguous().view(bs * n_slice_per_c * 7, 512)
        feat2, _ = self.lstm2(feat)

        return self.head(feat1), self.head2(feat2[:, 0])

In [23]:
# m = TimmModelType2(backbone)
# n_sequence = 7 * n_slice_per_c
# m(torch.rand(2, n_sequence, in_chans, image_size, image_size)).shape
# #     m(torch.rand(2, n_sequence, in_chans, image_size, image_size)).shape => torch.Size([2, 15])

In [24]:
# draw_graph(m, input_data = torch.rand(1, 3, 128,128,128), expand_nested=True, save_graph=True).visual_graph

# Loss & Metric

In [25]:
bce = nn.BCEWithLogitsLoss(reduction='none')


def criterion(logits, targets, activated=False):
    if activated:
        losses = nn.BCELoss(reduction='none')(logits.view(-1), targets.view(-1))
            # .view(-1) => return a single dimension tensor.
            # .view() => Returns a new tensor with the same data as the self tensor but of a different shape.
    else:
        losses = bce(logits.view(-1), targets.view(-1))
    losses[targets.view(-1) > 0] *= 2.
         # losses[targets.view(-1) > 0] => keeping those indices values from losses, where the targets are '> 0'.
         # losses[targets.view(-1) > 0] *= 2. =>   losses[targets.view(-1) > 0] = losses[targets.view(-1) > 0] * 2.
    norm = torch.ones(logits.view(-1).shape[0]).to(device)
    norm[targets.view(-1) > 0] *= 2
    return losses.sum() / norm.sum()

# Train & Valid func

In [26]:
def mixup(input, truth, clip=[0, 1]):
    indices = torch.randperm(input.size(0))
    shuffled_input = input[indices]
    shuffled_labels = truth[indices]

    lam = np.random.uniform(clip[0], clip[1])
    input = input * lam + shuffled_input * (1 - lam)
    return input, truth, shuffled_labels, lam


def train_func(model, loader_train, optimizer, scaler=None):    
    model.train()
    train_loss = []
    train_loss1 = []
    train_loss2 = []
    bar = tqdm(loader_train)
    for images, targets in bar:
        optimizer.zero_grad()
        images = images.cuda()
        targets = targets.cuda()
        
        do_mixup = False
        if random.random() < p_mixup:
            do_mixup = True
            images, targets, targets_mix, lam = mixup(images, targets)

        with amp.autocast():            
            logits, logits2 = model(images)
                # logits => tensor([[-1.1621],[-0.7876],[-0.7744],[-0.6548],[-0.8032], ...., [-0.5107], [-0.0616]], device='cuda:0',...)  
                # logits2 => tensor([[0.3247]], device='cuda:0',..)
                
                # targets.max() => tensor(1., device='cuda:0')
                # targets.max(1) => torch.return_types.max(values=tensor([1.], device='cuda:0'),indices=tensor([0], device='cuda:0'))
                # targets.max(1).values => tensor([1.], device='cuda:0')  
                # targets.max(dim=1) => Returns a namedtuple (values, indices) where values is the maximum value of each row of the input tensor in the given dimension dim=1. 

            loss1 = criterion(logits, targets)
            loss2 = criterion(logits2, targets.max(1).values)
                # loss1 => tensor(0.7388, device='cuda:0', grad_fn=<DivBackward0>) 
                # loss2 => tensor(0.7343, device='cuda:0', grad_fn=<DivBackward0>)
            loss = (loss1 * lw[0] + loss2 * lw[1]) / sum(lw)
            if do_mixup:
                loss11 = criterion(logits, targets_mix)
                loss22 = criterion(logits2, targets_mix.max(1).values)
                loss = loss * lam  + (loss11 * lw[0] + loss22 * lw[1]) / sum(lw) * (1 - lam)
        train_loss1.append(loss1.item())
        train_loss2.append(loss2.item())
        train_loss.append(loss.item())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

#         bar.set_description(f'smth:{np.mean(train_loss1[-30:]):.4f} {np.mean(train_loss2[-30:]):.4f}')

    return np.mean(train_loss)


def valid_func(model, loader_valid):
    model.eval()
    valid_loss = []
    valid_loss1 = []
    valid_loss2 = []
    outputs = []
    bar = tqdm(loader_valid)
    with torch.no_grad():
        for images, targets in bar:
            images = images.cuda()
            targets = targets.cuda()

            logits, logits2 = model(images)
            loss1 = criterion(logits, targets)
            loss2 = criterion(logits2, targets.max(1).values)
            loss = (loss1 + loss2) / 2.
            valid_loss1.append(loss1.item())
            valid_loss2.append(loss2.item())
            valid_loss.append(loss.item())
#             bar.set_description(f'smth:{np.mean(valid_loss1[-30:]):.4f} {np.mean(valid_loss2[-30:]):.4f}')

    return np.mean(valid_loss)



In [27]:
# rcParams['figure.figsize'] = 20, 2
# optimizer = optim.AdamW(m.parameters(), lr=init_lr)
# scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs, eta_min=eta_min)

# lrs = []
# for epoch in range(1, n_epochs+1):
#     scheduler_cosine.step(epoch-1)
#     lrs.append(optimizer.param_groups[0]["lr"])
# plt.plot(range(len(lrs)), lrs)

# Training

In [28]:
def run(fold):

    log_file = os.path.join(log_dir, f'{kernel_type}.txt')
    model_file = os.path.join(model_dir, f'{kernel_type}_fold{fold}_best.pth')

    train_ = df[df['fold'] != fold].reset_index(drop=True)
    valid_ = df[df['fold'] == fold].reset_index(drop=True)
    dataset_train = CLSDataset(train_, 'train', transform=transforms_train)
    dataset_valid = CLSDataset(valid_, 'valid', transform=transforms_valid)
    loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
    loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    model = TimmModelType2(backbone, pretrained=True)
    model = model.to(device)

    # if not first run, load previous model
    fold_l = 2
    load_model_file = os.path.join(model_dir_seg, f'{kernel_type}_fold{fold_l}_best.pth')
    sd = torch.load(load_model_file)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)    
    
    optimizer = optim.AdamW(model.parameters(), lr=init_lr)
    scaler = torch.cuda.amp.GradScaler() if use_amp else None
    from_epoch = 0
    metric_best = np.inf
    loss_min = np.inf

    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, n_epochs, eta_min=eta_min)

#     print(len(dataset_train), len(dataset_valid))

    for epoch in range(1, n_epochs+1):
        scheduler_cosine.step(epoch-1)
        if epoch < from_epoch + 1:
            print(logs[epoch-1])
            continue

        print(time.ctime(), 'Epoch:', epoch)

        train_loss = train_func(model, loader_train, optimizer, scaler)
        valid_loss = valid_func(model, loader_valid)
        metric = valid_loss

        content = time.ctime() + ' ' + f'Fold {fold}, Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.5f}, valid loss: {valid_loss:.5f}, metric: {(metric):.6f}.'
        print(content)
        with open(log_file, 'a') as appender:
            appender.write(content + '\n')

        if metric < metric_best:
            print(f'metric_best ({metric_best:.6f} --> {metric:.6f}). Saving model ...')
#             if not DEBUG:
            torch.save(model.state_dict(), model_file)
            metric_best = metric

#         # Save Last
#         if not DEBUG:
#             torch.save(
#                 {
#                     'epoch': epoch,
#                     'model_state_dict': model.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),
#                     'scaler_state_dict': scaler.state_dict() if scaler else None,
#                     'score_best': metric_best,
#                 },
#                 model_file.replace('_best', '_last')
#             )

    del model
    torch.cuda.empty_cache()
    _ = gc.collect()


In [29]:
# run(0)
# run(1)
run(2)
# run(3)
# run(4)

Mon Feb 27 08:07:20 2023 Epoch: 1


100%|█████████████████████████████████████████| 403/403 [01:39<00:00,  4.05it/s]


Mon Feb 27 08:20:13 2023 Fold 2, Epoch 1, lr: 0.0000002, train loss: 0.24960, valid loss: 0.31153, metric: 0.311531.
metric_best (inf --> 0.311531). Saving model ...
Mon Feb 27 08:20:14 2023 Epoch: 2


100%|█████████████████████████████████████████| 403/403 [01:39<00:00,  4.04it/s]


Mon Feb 27 08:32:32 2023 Fold 2, Epoch 2, lr: 0.0000002, train loss: 0.24179, valid loss: 0.31035, metric: 0.310353.
metric_best (0.311531 --> 0.310353). Saving model ...
Mon Feb 27 08:32:32 2023 Epoch: 3


100%|█████████████████████████████████████████| 403/403 [01:39<00:00,  4.06it/s]


Mon Feb 27 08:44:48 2023 Fold 2, Epoch 3, lr: 0.0000002, train loss: 0.24481, valid loss: 0.31267, metric: 0.312668.
Mon Feb 27 08:44:48 2023 Epoch: 4


 43%|█████████████████                       | 689/1615 [04:35<06:09,  2.50it/s]


KeyboardInterrupt: 